# Visualize Hurricane ECMWF data including track

This is an interactive page to view high resolution (HRES) simulation data done by the European Centre for Medium-Range Weather Forecasts (ECMWF). These data are downloaded locally in the lab at \\139.191.244.110\h\ECMWH\
The code below reads the local database and displays the data using as background the earth basemap. 

First let's import some necessary python modules

numpy for array manipulation
Ipython widgets for the interactive options
matplotlib for plotting
datetime for reading the time attribute of the data
Basemap for maps
pygrib for reading the grib files provided by ECMWF
glob for filtering out the needed files within the local folder
string for string manipulations

Finaly matplotlib inline is used to display the graph within the page, otherwise it creates a separate window.

In [1]:
import numpy as np
from ipywidgets import *
import matplotlib.pyplot as plt
import datetime
from mpl_toolkits.basemap import Basemap, shiftgrid
import pygrib
import glob
import string

In [2]:
%matplotlib notebook

## Flow field data

These data are located at the folder 'grib' folowed by the year/month/day subfolders and include the 10m U,V velocities and the msl pressure.
Below we define the basic functions that provides the data by passing the required data as arguments, namely year, month, day, hour of the corresponding data for visualization. The function getdata will parse the folder and retrieve the data. Note that there are 2 options for hours =(0,12). 

In [3]:
def getdata(yyyy=2015,mm=6,dd=3,hh=0):
    yyyy=np.int(yyyy)
    mm=np.int(mm)
    dd=np.int(dd)
    hh=np.int(hh) # This variable is passed as string and needs to become integer to be used later
    # specify date to plot.
    date = datetime.datetime(yyyy,mm,dd,hh)
    
    # set PATH of the database.
    PATHbase="/mnt/ECMWF/grib/"  # Local mapping location for the above network drive
    PATH=PATHbase+"%04i/%02i/%02i/" % (yyyy,mm,dd)

    dpath=glob.glob(PATH+"*%04i%02i%02i.%02i.tropical_cyclone.grib" % (yyyy,mm,dd,hh))
   
    try:
        data = pygrib.open(dpath[0])
    except:
        print 'no available data in ', PATH
        return 
    
    pd=data[1]
    ud=data[2]
    vd=data[3]
    # read lats,lons
    # reverse latitudes so they go from south to north.
    latitudes = pd.latlons()[0].T[0][::-1]
    longitudes = pd.latlons()[1][0]
    # get sea level pressure and 10-m wind data.
    # mult slp by 0.01 to put in units of hPa
    slpin = 0.01*pd.values[:].squeeze()
    uin = ud.values[:].squeeze()
    vin = vd.values[:].squeeze()


    # add cyclic points manually (could use addcyclic function)
    slp= np.zeros((slpin.shape[0],slpin.shape[1]+1),np.float64)
    slp[:,0:-1] = slpin[::-1]; slp[:,-1] = slpin[::-1,0]
    u= np.zeros((uin.shape[0],uin.shape[1]+1),np.float64)
    u[:,0:-1] = uin[::-1]; u[:,-1] = uin[::-1,0]
    v= np.zeros((vin.shape[0],vin.shape[1]+1),np.float64)
    v[:,0:-1] = vin[::-1]; v[:,-1] = vin[::-1,0]

    longitudes=np.append(longitudes,360.)
    return date, longitudes,latitudes,slp,u,v

### Cyclone tracking data

In order to view the hurricane we need to get the track from the ECMWF data. The track is provided in the name with the cyclone name and date included produced every 12 hours. These are stored in folder 'bufr' and reference the name of the tropical cyclone and are in bufr format. We can parse the folder to see the available storms for seach year



and retrieve the (date,lon,lat) of the bulletin. Thus we define the function below where the name of the strom is given as input and the date and location is returned.

In [4]:
def TC(yyyy=2015):

    tr=glob.glob('/mnt/ECMWF/bufr/%04i/*ECMF*tropical_cyclone_track_*' %(yyyy))


    name,btime,lon,lat=[],[],[],[]
    for i in range(np.size(tr)):
        a=tr[i].split('_')
        t=a[4]
        lon.append(np.float(a[-3].replace('p','.').translate(None,string.letters)))
        lat.append(np.float(a[-2].replace('p','.').translate(None,string.letters)))
        btime.append(str(datetime.datetime.strptime(t,'%Y%m%d%H%M%S')))
        name.append(a[8])

    return np.array([name,btime,lon,lat])


We can display the available TC like

In [5]:
res=TC(2016)
#res.sort()
tcnames=[]
for g in res[0]:
    if not g in tcnames: tcnames.append(g)
        
tcnames

['ULA',
 '12U',
 'ZENA',
 '06P',
 'ONE',
 '18P',
 'PALI',
 '04S',
 'DAYA',
 '08S',
 'FANTALA',
 '09U',
 'URIAH',
 'ALEX',
 'EMERAUDE',
 'VICTOR',
 '01B',
 'ROANU',
 '03S',
 'EX-URIAH',
 'CORENTIN',
 'WINSTON',
 'EX-EMERAUD',
 'EX-FANTALA',
 'AMOS',
 'EX-CORENTI',
 'YALO',
 '14P',
 'TWO',
 '08U',
 '17S',
 'BONNIE',
 'STAN',
 '07S',
 'NINE',
 '10U',
 'TATIANA',
 '14U',
 '16P',
 '20P',
 '07P']

Selecting the TC we want to see we can give the mean values of the storm's path as the view origin of the basemap. First we create a function to gather the data for the specific TC

In [6]:
def track(name):
    
    btime,lon,lat=[],[],[]
    for g in res.T: # transpose res
        if name in g[0]:
            btime.append(g[1])
            lon.append(g[2])
            lat.append(g[3])
    
    hdat=np.array([btime,lon,lat])
    ar=np.argsort(hdat) # sort with time
    return hdat[:,ar[0,:]]

In [7]:
tcname=tcnames[10]

In [8]:
hdat=track(tcname)
print tcname
hdat.T

FANTALA


array([['2016-04-12 00:00:00', '70.3', '-13.2'],
       ['2016-04-12 12:00:00', '68.8', '-13.2'],
       ['2016-04-13 00:00:00', '67.8', '-13.0'],
       ['2016-04-13 12:00:00', '66.0', '-12.9'],
       ['2016-04-14 00:00:00', '64.3', '-12.6'],
       ['2016-04-14 12:00:00', '62.5', '-12.4'],
       ['2016-04-15 00:00:00', '60.4', '-12.3'],
       ['2016-04-15 12:00:00', '58.8', '-12.5'],
       ['2016-04-16 00:00:00', '57.2', '-12.4'],
       ['2016-04-16 12:00:00', '55.3', '-12.0'],
       ['2016-04-17 00:00:00', '53.5', '-11.1'],
       ['2016-04-17 12:00:00', '51.7', '-10.4'],
       ['2016-04-18 00:00:00', '50.2', '-9.5'],
       ['2016-04-18 12:00:00', '49.3', '-9.3'],
       ['2016-04-19 00:00:00', '49.9', '-9.3'],
       ['2016-04-19 12:00:00', '50.4', '-9.4'],
       ['2016-04-20 00:00:00', '51.9', '-10.0'],
       ['2016-04-20 12:00:00', '53.5', '-11.0'],
       ['2016-04-21 00:00:00', '54.9', '-12.1'],
       ['2016-04-21 12:00:00', '56.3', '-12.6'],
       ['2016-04-22 00:0

In [9]:
# get the centre location 
lonc=hdat[1].astype(float).mean()
latc=hdat[2].astype(float).mean()

The function for visualizing the data in interactive mode is

In [10]:
def viewdata(latc=60,lonc=-60):
    
   
    lons, lats = np.meshgrid(longitudes,latitudes)

    # make orthographic basemap.
    m = Basemap(resolution='c',projection='ortho',lat_0=latc,lon_0=lonc)
    # create figure, add axes
    fig = plt.figure(figsize=(10,8))
    ax = fig.add_axes([0.1,0.1,0.8,0.8])
    # set desired contour levels.
    clevs = np.arange(960,1061,5)

    # compute native x,y coordinates of grid.
    x, y = m(lons, lats)
    # define parallels and meridians to draw.
    parallels = np.arange(-80.,90,20.)
    meridians = np.arange(0.,360.,20.)
    # plot SLP contours.
    CS1 = m.contour(x,y,slp,clevs,linewidths=0.5,colors='k',animated=True)
    CS2 = m.contourf(x,y,slp,clevs,cmap=plt.cm.RdBu_r,animated=True)
    # plot wind vectors on projection grid.
    # first, shift grid so it goes from -180 to 180 (instead of 0 to 360
    # in longitude).  Otherwise, interpolation is messed up.
    ugrid,newlons = shiftgrid(180.,u,longitudes,start=False)
    vgrid,newlons = shiftgrid(180.,v,longitudes,start=False)
    # transform vectors to projection grid.
    uproj,vproj,xx,yy = \
    m.transform_vector(ugrid,vgrid,newlons,latitudes,31,31,returnxy=True,masked=True)
    # now plot.
    Q = m.quiver(xx,yy,uproj,vproj,scale=700)
    # make quiver key.
    qk = plt.quiverkey(Q, 0.1, 0.1, 20, '20 m/s', labelpos='W')
    # draw coastlines, parallels, meridians.
    m.drawcoastlines(linewidth=1.5)
    m.drawparallels(parallels)
    m.drawmeridians(meridians)
    # add colorbar
    cb = m.colorbar(CS2,"bottom", size="5%", pad="2%")
    cb.set_label('hPa')
    # set plot title
    ax.set_title('SLP and Wind Vectors (10m) '+str(date))
    plt.show()

so now we call the first one for retriving the data. Note that this is not interactive so one has to set the values in the function arguments below. We can provide the date of interest based on the TC we investigate e.g.

In [11]:
dstamp=[]
for date,lon,lat in hdat.T:
  dstamp.append(datetime.datetime.strptime(date,'%Y-%m-%d %H:%M:%S'))

dstamp.sort()
dstamp

[datetime.datetime(2016, 4, 12, 0, 0),
 datetime.datetime(2016, 4, 12, 12, 0),
 datetime.datetime(2016, 4, 13, 0, 0),
 datetime.datetime(2016, 4, 13, 12, 0),
 datetime.datetime(2016, 4, 14, 0, 0),
 datetime.datetime(2016, 4, 14, 12, 0),
 datetime.datetime(2016, 4, 15, 0, 0),
 datetime.datetime(2016, 4, 15, 12, 0),
 datetime.datetime(2016, 4, 16, 0, 0),
 datetime.datetime(2016, 4, 16, 12, 0),
 datetime.datetime(2016, 4, 17, 0, 0),
 datetime.datetime(2016, 4, 17, 12, 0),
 datetime.datetime(2016, 4, 18, 0, 0),
 datetime.datetime(2016, 4, 18, 12, 0),
 datetime.datetime(2016, 4, 19, 0, 0),
 datetime.datetime(2016, 4, 19, 12, 0),
 datetime.datetime(2016, 4, 20, 0, 0),
 datetime.datetime(2016, 4, 20, 12, 0),
 datetime.datetime(2016, 4, 21, 0, 0),
 datetime.datetime(2016, 4, 21, 12, 0),
 datetime.datetime(2016, 4, 22, 0, 0),
 datetime.datetime(2016, 4, 22, 12, 0),
 datetime.datetime(2016, 4, 23, 0, 0),
 datetime.datetime(2016, 4, 23, 12, 0),
 datetime.datetime(2016, 4, 24, 0, 0),
 datetime.dat

In [12]:
#pich date
adate=dstamp[10]

In [13]:
date,longitudes,latitudes,slp,u,v=getdata(yyyy=adate.year, mm=adate.month, dd=adate.day, hh=adate.hour)

In [14]:
interact_manual(viewdata, lonc=lonc, latc=latc)

Widget Javascript not detected.  It may not be installed properly. Did you enable the widgetsnbextension? If not, then run "jupyter nbextension enable --py --sys-prefix widgetsnbextension"


<function __main__.viewdata>

The track of the hurricane can be super imposed on the figure above by modifing the function. The track until the date corresponding to the flow field shown is depicted. The forecast is not included.

In [15]:
def viewdata2(latc=60,lonc=-60):
    
    latc=np.float(latc)
    lonc=np.float(lonc)

    
    lons, lats = np.meshgrid(longitudes,latitudes)

    # make orthographic basemap.
    m = Basemap(resolution='c',projection='ortho',lat_0=latc,lon_0=lonc)
    # create figure, add axes
    fig = plt.figure(figsize=(10,8))
    ax = fig.add_axes([0.1,0.1,0.8,0.8])
    # set desired contour levels.
    clevs = np.arange(960,1061,5)

    # compute native x,y coordinates of grid.
    x, y = m(lons, lats)
    # define parallels and meridians to draw.
    parallels = np.arange(-80.,90,20.)
    meridians = np.arange(0.,360.,20.)
    # plot SLP contours.
    CS1 = m.contour(x,y,slp,clevs,linewidths=0.5,colors='k',animated=True)
    CS2 = m.contourf(x,y,slp,clevs,cmap=plt.cm.RdBu_r,animated=True)
    # plot wind vectors on projection grid.
    # first, shift grid so it goes from -180 to 180 (instead of 0 to 360
    # in longitude).  Otherwise, interpolation is messed up.
    ugrid,newlons = shiftgrid(180.,u,longitudes,start=False)
    vgrid,newlons = shiftgrid(180.,v,longitudes,start=False)
    # transform vectors to projection grid.
    uproj,vproj,xx,yy = \
    m.transform_vector(ugrid,vgrid,newlons,latitudes,31,31,returnxy=True,masked=True)
    # now plot.
    Q = m.quiver(xx,yy,uproj,vproj,scale=700)
    # make quiver key.
    qk = plt.quiverkey(Q, 0.1, 0.1, 20, '20 m/s', labelpos='W')
    
    #plot the track of the hurricane up to the date choosen
    #define date contraint
    ii=np.argwhere(hdat[0]<str(date)).max()+1 #get the maximum index and adjust for numpy array indexing 
    xh=hdat[1][:ii].astype(float) 
    yh=hdat[2][:ii].astype(float)
    # compute native xh,yh coordinates of grid.
    xh, yh = m(xh, yh)
    m.plot(xh,yh,'r--x')
    
    # draw coastlines, parallels, meridians.
    m.drawcoastlines(linewidth=1.5)
    m.drawparallels(parallels)
    m.drawmeridians(meridians)
    # add colorbar
    cb = m.colorbar(CS2,"bottom", size="5%", pad="2%")
    cb.set_label('hPa')
    # set plot title
    ax.set_title('SLP and Wind Vectors (10m) '+str(date))
    plt.show()

In [16]:
interact_manual(viewdata2, lonc=lonc, latc=latc)

Widget Javascript not detected.  It may not be installed properly. Did you enable the widgetsnbextension? If not, then run "jupyter nbextension enable --py --sys-prefix widgetsnbextension"


<function __main__.viewdata2>

## TC track forecast

The ECMWF storm files are bufr files and include additional information such as pressure, wind speed at 10m and forecast. In order to access this info we need to import the python bufr modules and the bunch module for convinience. 

In [17]:
from pybufr_ecmwf.bufr import BUFRReader
from pybufr_ecmwf.raw_bufr_file import RawBUFRFile
from pybufr_ecmwf.bufr_interface_ecmwf import BUFRInterfaceECMWF

from bunch import Bunch


We can define a function that reads this files and return all necessary info, like this....

In [18]:
from readbufr import * 

In [19]:
trackdata=read_bufr(tcnames[10])

In [20]:
times=sorted(trackdata.keys())
times

['201604120000',
 '201604121200',
 '201604130000',
 '201604131200',
 '201604140000',
 '201604141200',
 '201604150000',
 '201604151200',
 '201604160000',
 '201604161200',
 '201604170000',
 '201604171200',
 '201604180000',
 '201604181200',
 '201604190000',
 '201604191200',
 '201604200000',
 '201604201200',
 '201604210000',
 '201604211200',
 '201604220000',
 '201604221200',
 '201604230000',
 '201604231200',
 '201604241200']

In [21]:
trackdata[times[10]].plons

array([  5.37000000e+01,   5.30000000e+01,   5.21000000e+01,
         5.16000000e+01,   5.10000000e+01,   5.04000000e+01,
         5.00000000e+01,   5.00000000e+01,   5.02000000e+01,
         5.04000000e+01,   5.11000000e+01,   5.20000000e+01,
         5.28000000e+01,   5.37000000e+01,   5.45000000e+01,
         5.52000000e+01,   5.59000000e+01,   5.65000000e+01,
         5.73000000e+01,   5.83000000e+01,   5.93000000e+01,
         6.00000000e+01,   6.07000000e+01,   6.13000000e+01,
         6.14000000e+01,   6.14000000e+01,   6.14000000e+01,
         1.70000000e+38,   1.70000000e+38,   1.70000000e+38,
         6.06000000e+01,   6.01000000e+01,   1.70000000e+38,
         1.70000000e+38,   5.94000000e+01,   5.94000000e+01,
         1.70000000e+38,   1.70000000e+38,   1.70000000e+38,
         6.07000000e+01])

In [22]:
trackdata[times[10]].t

array([   0.,    6.,   12.,   18.,   24.,   30.,   36.,   42.,   48.,
         54.,   60.,   66.,   72.,   78.,   84.,   90.,   96.,  102.,
        108.,  114.,  120.,  126.,  132.,  138.,  144.,  150.,  156.,
        162.,  168.,  174.,  180.,  186.,  192.,  198.,  204.,  210.,
        216.,  222.,  228.,  234.])

We can now create an interactive function for displaying the tracks.

In [23]:
def viewtrack(index):
    t=trackdata[index].t
    lons=trackdata[index].plons
    lats=trackdata[index].plats    
    
    # Note that
    # first lon,lat = TC center
    # second lon,lat = Lowest pressure loc & first p_msl is the value
    # third  lon,lat = maximum wind speed at 10m loc & first u10  is the value
    # then lon,lat in pairs for forecasted loc of TC center and loc of maximum wind

    if lons.size > 3:
        lonp=np.append(lons[1],lons[3::2])
        latp=np.append(lats[1],lats[3::2])
        tp=np.append(0.,t)  
        
        m1=np.abs(lonp)<180.
        m2=np.abs(latp)<90.
        
        if (m1 != m2).any() :
            print 'problem'
        else:
            lonp=lonp[m1]
            latp=latp[m1]
            tp=tp[m1]
    
    else:
        
        lonp=[lons[0],lons[0]]
        latp=[lats[0],lats[0]]
        tp=[0.,0.]
                
    latc=np.mean(latp)
    lonc=np.mean(lonp)
        
    llcrnrlon=np.min(lonp)-15.
    llcrnrlat=np.min(latp)-15.
    urcrnrlon=np.max(lonp)+15
    urcrnrlat=np.max(latp)+15
    lon_0=np.mean([llcrnrlon,urcrnrlon])
    
    # Lambert Conformal Conic map.
    m = Basemap(llcrnrlon=llcrnrlon,llcrnrlat=llcrnrlat,urcrnrlon=urcrnrlon,urcrnrlat=urcrnrlat,
            projection='cyl',lat_1=llcrnrlat,lat_2=urcrnrlat,lon_0=-60.,
            resolution ='l',area_thresh=1000.)
  # create figure, add axes
    fig = plt.figure(figsize=(10,8))
    ax = fig.add_axes([0.1,0.1,0.8,0.8])
    # set desired contour levels.
    clevs = np.arange(960,1061,5)
        
    # define parallels and meridians to draw.
    parallels = np.arange(-80.,90,20.)
    meridians = np.arange(0.,360.,20.)

    # plot track    
    xh, yh = m(lons, lats)
    m.plot(xh,xh,'r--o')
    
    for label , xp, yp in zip(t,lons,lats):
        xmp,ymp=m(xp,yp)
        if (xmp<400.) & (ymp<100.):
          plt.annotate('%02i' %(label),xy=(xmp,ymp), xytext=(10,0), textcoords='offset points', size='large',color='k')


    
    
    # draw coastlines, parallels, meridians.
    m.drawcoastlines(linewidth=1.5)
    m.drawparallels(parallels)
    m.drawmeridians(meridians)
    
    # set plot title
    ax.set_title('track of TC '+tcname+' at time '+np.str(index))
    plt.show()
        
    

In [24]:
interact_manual(viewtrack, index=times)

Widget Javascript not detected.  It may not be installed properly. Did you enable the widgetsnbextension? If not, then run "jupyter nbextension enable --py --sys-prefix widgetsnbextension"


<function __main__.viewtrack>

Overlay the vector field from ECMWF data

In [25]:
def viewtrackf(index):
    t=trackdata[index].t
    tlons=trackdata[index].plons
    tlats=trackdata[index].plats    
    
    
    
    # Note that
    # first lon,lat = TC center
    # second lon,lat = Lowest pressure loc & first p_msl is the value
    # third  lon,lat = maximum wind speed at 10m loc & first u10  is the value
    # then lon,lat in pairs for forecasted loc of TC center and loc of maximum wind
    # IF there is no prediction the only data are the TC center

    if t.any():
        lonp=np.append(lons[1],lons[3::2])
        latp=np.append(lats[1],lats[3::2])
        tp=np.append(0.,t)

        m1=np.abs(lonp)<180.
        m2=np.abs(latp)<90.

        if (m1 != m2).any() :
            print 'problem'
        else:
            lonp=lonp[m1]
            latp=latp[m1]
            tp=tp[m1]
        
        
        latc=np.mean(latp)
        lonc=np.mean(lonp)
    else:
        lonp=np.array([lons[0]])
        latp=np.array([lats[0]])
        tp=np.array([0.])

    latc=np.mean(latp)
    lonc=np.mean(lonp)

    
    
    llcrnrlon=np.min(lonp)-15.
    llcrnrlat=np.min(latp)-15.
    urcrnrlon=np.max(lonp)+15
    urcrnrlat=np.max(latp)+15
    lon_0=np.mean([llcrnrlon,urcrnrlon])
    
    # get the ECMWF data 
    # first get the date from index
    dv=datetime.datetime.strptime(index,'%Y%m%d%H%M')
    date,longitudes,latitudes,slp,u,v=getdata(yyyy=dv.year, mm=dv.month, dd=dv.day, hh=dv.hour)    

    def view():
    
      # Lambert Conformal Conic map.
      m = Basemap(llcrnrlon=llcrnrlon,llcrnrlat=llcrnrlat,urcrnrlon=urcrnrlon,urcrnrlat=urcrnrlat,
            projection='cyl',lat_1=llcrnrlat,lat_2=urcrnrlat,lon_0=-60.,
            resolution ='l',area_thresh=1000.)
    
     # create figure, add axes
      fig = plt.figure()#figsize=(10,8))
    # set desired contour levels.
      clevs = np.arange(960,1061,5)
        
    # define parallels and meridians to draw.
      parallels = np.arange(-80.,90,20.)
      meridians = np.arange(0.,360.,20.)

    # plot track    
      xh, yh = m(lonp, latp)
      for x1,x2 in zip(xh,yh):
        if (x1<400.) & (x2<100.):
           m.plot(x1,x2,'r--o')
    
      for label , xp, yp in zip(t,lonp,latp):
        xmp,ymp=m(xp,yp)
        if (xmp<400.) & (ymp<100.):
          plt.annotate('%02i' %(label),xy=(xmp,ymp), xytext=(10,0), textcoords='offset points', size='large',color='k')


    # Evaluate data for vector field plot
      lons, lats = np.meshgrid(longitudes,latitudes)
    # compute native x,y coordinates of grid.
      x, y = m(lons, lats)
      CS1 = m.contour(x,y,slp,clevs,linewidths=0.5,colors='k',animated=True)
      CS2 = m.contourf(x,y,slp,clevs,cmap=plt.cm.RdBu_r,animated=True)
    # plot wind vectors on projection grid.
    # first, shift grid so it goes from -180 to 180 (instead of 0 to 360
    # in longitude).  Otherwise, interpolation is messed up.
      ugrid,newlons = shiftgrid(180.,u,longitudes,start=False)
      vgrid,newlons = shiftgrid(180.,v,longitudes,start=False)
    # transform vectors to projection grid.
      uproj,vproj,xx,yy = \
      m.transform_vector(ugrid,vgrid,newlons,latitudes,31,31,returnxy=True,masked=True)
    # now plot.
      Q = m.quiver(xx,yy,uproj,vproj,scale=700)
    # make quiver key.
      qk = plt.quiverkey(Q, 0.1, 0.1, 20, '20 m/s', labelpos='W')

    
    # draw coastlines, parallels, meridians.
      m.drawcoastlines(linewidth=1.5)
      m.drawparallels(parallels)
      m.drawmeridians(meridians)
    
    # set plot title
      plt.title('track of TC '+tcname+' at time '+np.str(index))
      plt.show()
        
    view()

animate

In [ ]:
interact_manual(viewtrackf,index=times)

Now what we need is choose the TC and then display for any given day the track and forecast on top of the current flow field. An animation of the forecast would also be nice. 

An development upgrate based on the above is in [HECMWF2](HECMWF2.ipynb) while a compact form including the more efficient way of reading grib files ([Reading Grib Files](Reading GRIB Files.ipynb)) is here [CompactViewer](HOP.ipynb).